# Using SARA API

In [2]:
import webbrowser
import urllib.request, json 
from datetime import date, timedelta
from math import sin, cos, sqrt, atan2, radians
from shapely.wkt import loads as load_wkt

In [126]:
class sara():
    
    def __init__ (self, geometry, dataRange = 10, completionDate = date.today()):
        self.email = 'mlsa.unimelb.au@gmail.com'
        self.password = 'mlsa12345'
        self.startDate = completionDate - timedelta(days=10)
        self.completionDate = completionDate
        self.geometry = geometry
        self.centroid = load_wkt(geometry).centroid
    
    def get_url(self):
        self.link = "https://copernicus.nci.org.au/sara.server/1.0/api/collections/S2/search.json?_pretty=1&startDate={}&completionDate={}&geometry={}&processingLevel=L2A".format(self.startDate,self.completionDate,self.geometry)
        self.link = self.link.replace(" ","")
        return self.link
    
    def get_distance(self,lon2,lat2):
        # approximate radius of earth in km
        R = 6373.0
        lon1 = self.centroid.x
        lat1= self.centroid.y

        lat1 = radians(lat1)
        lon1 = radians(lon1)
        lat2 = radians(lat2)
        lon2 = radians(lon2)

        dlon = lon2 - lon1
        dlat = lat2 - lat1

        a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
        c = 2 * atan2(sqrt(a), sqrt(1 - a))

        distance = R * c # in km
        
        return distance

    def get_json(self):
        with urllib.request.urlopen(self.link) as url:
            self.data = json.loads(url.read().decode())
            count = self.data['properties']['totalResults']
            download_links = []
            for i in range(count):
                if self.data['features'][i]['properties']['cloudCover'] <= 20:
                    feature_centroid = self.data['features'][i]['properties']['centroid']['coordinates']
                    if self.get_distance(feature_centroid[0], feature_centroid[1]) <= 100:
                        download_links.append(self.data['features'][i]['properties']['services']['download']['url'])
        self.download_links = download_links
        return self.download_links
    
    def download_from_link(self):
        download_link = self.download_links[0]
        ! curl -n -L -O -J -k -u {self.email}:{self.password} {download_link}
        
                


In [127]:
test = sara(geometry = 'POLYGON((149.03 -35.18,149.23 -35.18,149.23 -35.38,149.03 -35.38,149.03 -35.18))')
test.get_url()
test.get_json()

['https://copernicus.nci.org.au/sara.server/1.0/collections/S2/0a84458f-23d5-5698-acea-9ff1284a22ba/download',
 'https://copernicus.nci.org.au/sara.server/1.0/collections/S2/a3cd8983-dd81-503e-bd4d-cea0b707da4d/download',
 'https://copernicus.nci.org.au/sara.server/1.0/collections/S2/95e4bb41-c612-5210-b8bc-f650feea3582/download',
 'https://copernicus.nci.org.au/sara.server/1.0/collections/S2/46b3d073-5976-507a-8516-2ff82d7c1f1f/download',
 'https://copernicus.nci.org.au/sara.server/1.0/collections/S2/5df07522-7674-5be1-b5d1-85a0ae189cbe/download',
 'https://copernicus.nci.org.au/sara.server/1.0/collections/S2/b018c0b8-e607-5135-af4c-ffd7c9a5cf69/download',
 'https://copernicus.nci.org.au/sara.server/1.0/collections/S2/6fa54ee0-8ac7-517c-a9af-1ec7fdc3cbcf/download',
 'https://copernicus.nci.org.au/sara.server/1.0/collections/S2/b6ab70af-76a4-5bd3-afdf-eaf1f32389ba/download']

In [128]:
test.download_from_link()

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:--  0:00:03 --:--:--     0
100   336  100   336    0     0     50      0  0:00:06  0:00:06 --:--:--   120
  1 1009M    1 13.8M    0     0   420k      0  0:41:01  0:00:33  0:40:28  720k^C
